### 주식 정보 Crawling (from 네이버 금융)
- Ref. https://bigdata-doctrine.tistory.com/3

In [63]:
# 수집할 증권 데이터의 셀렉터
per_selector = "#_per"
pbr_selector = "#_pbr"
eps_selector = "#_eps"
dividend_selector = "#_dvr"

per_list = []
pbr_list = []
eps_list = []
dividend_list = []

# 수집할 주식 종목 리스트
ticker_list = ['267290', '117580', '004690', '017390', '034590']

In [64]:
ticker_list, per_list, pbr_list, eps_list, dividend_list

(['267290', '117580', '004690', '017390', '034590'], [], [], [], [])

In [65]:
# 크롤링을 사용하여 증권 데이터 수집하기

import requests
from bs4 import BeautifulSoup

# ticker = "005930"
for ticker in ticker_list:
    url = f"https://finance.naver.com/item/main.naver?code={ticker}"
    # headers={'User-agent':'Mozilla/5.0'}
    # 수집하는 주체가 봇이 아닌 유저임을 알리기 위함.
    # 이 코드를 삽입하지 않을 경우 봇으로 인지해 크롤링을 할 수 없습니다
    html = requests.get(url, headers={'User-agent':'Mozilla/5.0'})  
    soup = BeautifulSoup(html.text, "lxml")

    per = soup.select(per_selector)
    pbr = soup.select(pbr_selector)
    eps = soup.select(eps_selector)
    dividend = soup.select(dividend_selector)

    if not per:  # 리스트가 비어있을 경우 None으로 변환
        per = [None]
    for pe in per:
        if pe != None:
            per_text = pe.text
            per_text = per_text.replace(",", "")  # float변환을 위해 따옴표 제거
            per_list.append(float(per_text))
        else:
            per_text = pe
            per_list.append(per_text)

    if not pbr:  # 리스트가 비어있을 경우 None으로 변환
        pbr = [None]
    for pb in pbr:
        if pb != None:
            pbr_text = pb.text
            pbr_text = pbr_text.replace(",", "")  # float변환을 위해 따옴표 제거
            pbr_list.append(float(pbr_text))
        else:
            pbr_text = pb
            pbr_list.append(pbr_text)
            
    if not eps:  # 리스트가 비어있을 경우 None으로 변환
        eps = [None]
    for e in eps:
        if e != None:
            eps_text = e.text
            eps_text = eps_text.replace(",", "")  # float변환을 위해 따옴표 제거
            eps_list.append(float(eps_text))
        else:
            eps_text = e
            eps_list.append(eps_text)            
        
    if not dividend:  # 리스트가 비어있을 경우 None으로 변환
        dividend = [None]
    for d in dividend:
        if d != None:
            dividend_text = d.text
            dividend_text = dividend_text.replace(",", "")  # float변환을 위해 따옴표 제거
            dividend_list.append(float(dividend_text))
        else:
            dividend_text = d
            dividend_list.append(dividend_text)

In [66]:
ticker_list, per_list, pbr_list, eps_list, dividend_list

(['267290', '117580', '004690', '017390', '034590'],
 [4.96, 134.84, 5.82, 23.02, 9.49],
 [0.3, 0.71, 0.25, 0.25, 0.53],
 [4153.0, 62.0, 18490.0, 2941.0, 2734.0],
 [4.25, 2.99, 2.79, 2.59, 4.82])

In [67]:
from pykrx import stock

stock_names = []
for ticker in ticker_list:
    stock_names.append(stock.get_market_ticker_name(ticker))

In [68]:
stock_names

['경동도시가스', '대성에너지', '삼천리', '서울가스', '인천도시가스']

In [74]:
import pandas as pd

# 복수의 리스트를 하나의 데이터프레임으로 병합
# df_stock = pd.DataFrame(zip(ticker_list, stock_names, pers, pbrs, dividend_yields))
# df_stock.columns = ['Ticker', 'Stock Name', 'PER', 'PBR', 'Dividend Yield']
df_stock = pd.DataFrame({'Ticker' : ticker_list, 'Stock Name' : stock_names, 'PER' : per_list, 'PBR' : pbr_list, 'EPS' : eps_list, 'Dividend' : dividend_list})
df_stock 

,Ticker,Stock Name,PER,PBR,EPS,Dividend
0,267290,경동도시가스,4.96,0.30,4153.0,4.25
1,117580,대성에너지,134.84,0.71,62.0,2.99
2,004690,삼천리,5.82,0.25,18490.0,2.79
3,017390,서울가스,23.02,0.25,2941.0,2.59
4,034590,인천도시가스,9.49,0.53,2734.0,4.82
